&emsp;이번 포스팅에서는 파이썬 **Dask DataFrame**을 활용한 간단한 예제를 다뤄보도록 하겠습니다.

# 목차

---

# Dask DataFrame

<figure>
    <img src="./img/dask-dataframe.svg" width="300px">
</figure>

&emsp;Dask DataFrame은 위의 그림처럼 인덱스를 따라 분할된 여러 Pandas 데이터프레임으로 구성되어 있습니다. 따라서, Dask DataFrame은 Pandas API가 갖고있는 기능들을 지원합니다.

---

# Dask 클라이언트

&emsp;Dask 클라이언트(client)는 필요에 따라 실행해도 되고, 실행하지 않아도 됩니다. Dask 클라이언트는 연산 작업 현황에 대한 정보를 얻는데 유용한 대시보드(dashboard)를 제공합니다.

&emsp;아래 코드와 같이 클라이언트를 생성하면 대시보드에 대한 링크가 표시됩니다. 이 링크를 클릭해, 작업을 실행하는 동안 다른 화면 한쪽에 대시보드를 열어 두는 것이 좋습니다. 대시보드를 작업 화면과 동시에 보는 것은 분석 및 학습을 수행할 때 매우 유용합니다.

In [1]:
from dask.distributed import Client

client = Client(n_workers=4, threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 15.87 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59064,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 15.87 GiB
Comm: tcp://127.0.0.1:59083,Total threads: 1
Dashboard: http://127.0.0.1:59087/status,Memory: 3.97 GiB
Nanny: tcp://127.0.0.1:59067,


---

# 난수 데이터프레임 생성

&emsp;여기에서 사용할 데이터를 만들기 위해, 다음과 같은 특성을 가진 임의의 시계열 데이터를 생성합니다.  

1. 2000년 1월 한 달의 기간동안 매초마다의 레코드를 저장합니다.
2. 해당 월을 일 단위로 분할하여, 1일 치 레코드를 분할된 데이터프레임으로 만듭니다.
3. 데이터프레임의 인덱스는 datetime이며, 그 외에 name, id, x, y 열로 구성되어 있습니다.

&emsp;이 시계열 데이터는 약 240 MB 크기의 작은 데이터셋입니다. 더 큰 데이터셋으로 만들기 위해, [`dask.datasets.timeseries()`](https://docs.dask.org/en/stable/api.html#dask.datasets.timeseries) 인수를 설정하여 일 수(the number of days)를 늘리거나 데이터 포인트의 시간 간격을 줄일 수 있습니다. Dask DataFrame은 Pandas 데이터프레임과 다르게 연산이 필요할 때만 데이터 값들이 로드됩니다. 이러한 Dask의 특성을 *lazy*하다고 표현합니다. 따라서, 데이터 값이 노출되지 않고 줄임표(`...`)로 대체됩니다.

In [7]:
import dask

df = dask.datasets.timeseries(start="2000-01-01", end="2023-12-31")
df

,name,id,x,y
npartitions=8765,,,,
2000-01-01,string,int32,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2023-12-30,...,...,...,...
2023-12-31,...,...,...,...


&emsp;그래도 열 이름과 데이터 타입(dtype)은 확인할 수 있습니다.

In [8]:
df.dtypes

name    string[pyarrow]
id                int32
x               float64
y               float64
dtype: object

&emsp;아래 `.head()`와 같이 일부 작업을 하는 경우에는, 데이터 값이 자동으로 노출 됩니다.

In [9]:
df.head()

,name,id,x,y
timestamp,,,,
2000-01-01 00:00:00,Ursula,1001,0.495516,0.428047
2000-01-01 00:00:01,Patricia,1001,-0.337798,-0.598631
2000-01-01 00:00:02,Ray,965,-0.748407,-0.898091
2000-01-01 00:00:03,Charlie,955,0.767631,0.921614
2000-01-01 00:00:04,Wendy,941,-0.328476,-0.369255


---

# Dask에서 Pandas 작업하기

&emsp;Dask 데이터프레임은 대부분의 Pandas 데이터프레임과 동일한 방식으로 작업할 수 있습니다. 아래 예제는 Dask 데이터프레임에서 어떠한 조건을 기준으로 데이터를 분할한 다음, `x` 열에 있는 데이터의 표준 편차를 계산하는 방법을 보여줍니다.

In [10]:
df2 = df[df.y > 0]
df3 = df2.groupby("name").x.std()
df3

Dask Series Structure:
npartitions=1
    float64
        ...
Name: x, dtype: float64
Dask Name: sqrt, 10 graph layers

&emsp;`df3`의 데이터는 아직까지 실제로 연산되지는 않으므로 줄임표(`...`)로 표시됩니다. 연산 결과를 Pandas 데이터프레임 또는 시리즈(Pandas.Series)로 나타내길 원한다면, `.compute()`를 호출해야 합니다. 만약 위에서 `Client()`를 실행한 상태라면, 연산이 진행되는 동안 상태 페이지를 보고 진행 상황을 확인할 수 있습니다.

In [11]:
computed_df = df3.compute()
computed_df

name
Alice       0.577211
Bob         0.577371
Charlie     0.577433
Dan         0.577360
Edith       0.577350
Frank       0.577384
George      0.577253
Hannah      0.577352
Ingrid      0.577337
Jerry       0.577354
Kevin       0.577447
Laura       0.577477
Michael     0.577373
Norbert     0.577394
Oliver      0.577331
Patricia    0.577218
Quinn       0.577301
Ray         0.577351
Sarah       0.577387
Tim         0.577280
Ursula      0.577361
Victor      0.577541
Wendy       0.577428
Xavier      0.577356
Yvonne      0.577243
Zelda       0.577365
Name: x, dtype: float64

In [9]:
type(computed_df)

pandas.core.series.Series

&emsp;또다른 예시를 살펴보겠습니다. 아래의 예는 여러 개 열을 다른 방식으로 집계하는 작업을 나타낸 것입니다. 마찬가지로, 대시보드에 연산 진행 상황이 표시됩니다.

In [10]:
df4 = df.groupby("name").aggregate({"x": "sum", "y": "max"})
df4.compute()

,x,y
name,,
Wendy,77.738573,0.999994
Norbert,4.418316,0.999967
Jerry,-201.759701,0.999986
Frank,-78.946458,0.999989
Alice,91.564709,0.999995
Bob,389.110117,0.999993
Dan,47.803073,0.999999
Hannah,257.145433,0.999984
Oliver,61.472833,0.999992
